## X-ray diffraction

We begin by loading the libraries you will require later on for your calculations.

In [14]:
# import required libraries
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

Further you need to install the library [pyFAI](https://pyfai.readthedocs.io/en/stable/) that we will use in the remainder of this notebook.

In [15]:
# pip install ipywidgets --upgrade
# pip install jupyterlab
# pip install notebook

---
### Exercise 1

Go to the [pyFAI cookbook](https://pyfai.readthedocs.io/en/stable/usage/cookbook/calibration_with_jupyter.html) and **copy the different cells** into this notebook to try and recreate the calibration of the diffraction detector as described in the cookbook. **Discuss and describe** the different steps you are performing in your own words to make sure you understand the process.

**Hints:**
- you may need to select a different matplotlib interface (**nbagg**, **widget** or **inline**), depending on which environment you are using.
- when you get to the stage where you should select the rings on the detector, make sure that you **start at number 0** for the **innermost diffraction ring**.

---
#### Become familiar with pyFAI cookbook
_- solve the exercise beneath using markdown and/or code blocks -_

In [ ]:
from IPython.display import Video

Video("http://www.silx.org/pub/pyFAI/video/Calibration_Jupyter.mp4", width=800)


In [ ]:
for lib in ["jupyterlab", "notebook", "matplotlib", "ipympl", "ipywidgets"]:
    mod = __import__(lib)
    print(f"{lib:12s}:   {mod.__version__}")

In [ ]:
# The notebook interface (nbagg) is needed in jupyter-notebook while the widget is recommended for jupyer lab
# %matplotlib nbagg
# %matplotlib widget
# %matplotlib inline
# For the integration in the documentation, one uses `inline` to capture figures.
%matplotlib inline

import pyFAI
import pyFAI.test.utilstest
import fabio
from matplotlib.pyplot import subplots
from pyFAI.gui import jupyter
from pyFAI.gui.jupyter.calib import Calibration

print(f"PyFAI version {pyFAI.version}")

In [19]:
# Some parameters like the wavelength, the calibrant and the diffraction image:
wavelength = 
pilatus = pyFAI.detector_factory("")
AgBh = pyFAI.calibrant.CALIBRANT_FACTORY("")
AgBh.wavelength = 

# load some test data (requires an internet connection)
img = fabio.open(pyFAI.test.utilstest.UtilsTest.getimage("")).data

In [ ]:
# Simply display the scattering image:
jupyter.display()

In [ ]:
%matplotlib widget
calib = Calibration(, calibrant=, wavelength=, detector=)
# This displays the calibration widget:

# 1. Set the ring number (0-based value), below the plot
# 2. Pick the ring by right-clicking with the mouse on the image.
# 3. Restart at 1. for at least a second ring
# 4. Click refine to launch the calibration.

In [ ]:
# This is the calibrated geometry:
gr = calib.
print(gr)
print(f"Fixed parameters: {calib.}")
print(f"Cost function: {gr.}")

In [23]:
# re-extract all control points using the "massif" algorithm
calib.extract_cpt()

In [24]:
# remove the last ring since it is outside the flight-tube
calib.remove_grp(lbl="f")

In [ ]:
# Switch back to `inline` mode to capture the last plot
%matplotlib inline
calib.peakPicker.widget.fig.show()

In [ ]:
# Those are some control points: the last column indicates the ring number
calib.geoRef.data[::100]

In [ ]:
# This is the geometry with all rings defined:
gr = calib.
print(gr)
print(f"Fixed parameters: {calib.}")
print(f"Cost function: {gr.}")

In [ ]:
# Geometry refinement with some constrains: SAXS mode
# Here we enforce all rotation to be null and fit again the model:

gr. = gr. = gr. = 0
gr.refine3(fix=["rot1", "rot2", "rot3", "wavelength"])
print(gr)
print(f"Cost function = {gr.chi2()}")

In [ ]:
gr.save("jupyter.poni")
gr.get_config()

In [ ]:
# Create a "normal" azimuthal integrator (without fitting capabilities from the geometry-refinement object)
ai = pyFAI.load(gr)
ai

In [ ]:
# Display the integrated data to validate the calibration.
fig, ax = subplots(1, 2, figsize=(12, 6))
jupyter.plot1d(ai.integrate1d(img, 1000), calibrant=AgBh, ax=ax[0])
jupyter.plot2d(ai.integrate2d(img, 1000), calibrant=AgBh, ax=ax[1])
_ = ax[1].set_title("2D integration")

---
### Exercise 2

In this exercise, you will perform another detector calibration and then continue to integrate data from a recent experiment.  

In a **first step**, use the **code snippets** copied in the [pyFAI cookbook](https://pyfai.readthedocs.io/en/stable/usage/cookbook/calibration_with_jupyter.html) from **Exercise 1** to adapt the calibration process of the cookbook **only for relevant steps**. You can write a function or simply adapt the given code for the relevant parts only.

The mentioned diffraction experiment was performed at an **X-ray energy** of $87.1\,\text{keV}$. The detector calibration data is given in the file `Lab6-00001.tif` located in the **data folder**. 
To sucessfully perform the calibration, you have to use [Lanthanum hexaboride](https://en.wikipedia.org/wiki/Lanthanum_hexaboride) ``LaB6`` as **calibrant**, ``PerkinElmer`` ([Link]([detector](https://resources.perkinelmer.com/corporate/pdfs/downloads/bro_digitalxraydetectors.pdf))) as **detector**.

**Given Constants:**
- Speed of light, $ c = 3 \times 10^8 \, \text{m/s} $
- Planck's constant, $ h = 4.136 \times 10^{-15} \, \text{eV} \cdot \text{s} $
- Beam Energie, $ E_0 = 87.1 \, \text{keV} $

**Hint:**
- The imported ``Calibration`` Class from [pyFAI](https://pyfai.readthedocs.io/en/stable/) exspects wavelength given in m.

**Acknowledgments:**  
The data used in this exercise is courtesy of **Dr. Florian Wieland** at Helmholtz-Zentrum Hereon.

---
#### Solution
_- solve the exercise beneath using markdown and/or code blocks -_

In [32]:
# Import relevant modules
import fabio
import pyFAI
from pyFAI.calibrant import CALIBRANT_FACTORY
from matplotlib import pyplot as plt
from pyFAI.gui.jupyter.calib import Calibration
from pyFAI.gui import jupyter

In [33]:
# function to plot execute calibration after interactive
def calibration_results(calibrated_image_object, nbr_of_points=1000):
    """
    Calibrates the geometry for SAXS mode by extracting control points, refining the geometry,
    and validating the calibration through azimuthal integration of the provided image data.

    Parameters:
    calibrated_image_object: An object that contains the calibration data and methods for extraction and refinement.
    nbr_of_points: The number of points for azimuthal integration (default is 1000).

    Returns:
    integrator: An azimuthal integrator object created from the calibrated geometry.
    """
    calib = calibrated_image_object
    calib.  # re-extract all control points using the "massif" algorithm
    calib.  # Remove rings outside the flight-tube

    # Geometry after control points extraction
    geometry = calib.

    # Apply constraints for SAXS mode and refine
    geometry. = geometry. = geometry. = 0
    geometry.refine3(fix=["rot1", "rot2", "rot3", "wavelength"])
    print(geometry)
    print(f"Fixed parameters: {calib.}")
    print(f"Cost function = {geometry.}")

    # Create an azimuthal integrator from the calibrated geometry
    integrator = pyFAI.load()

    # Validate calibration by displaying integrated data
    fig, ax = plt.subplots(1, 2)
    jupyter.plot1d(
        integrator.integrate1d(, ), , ax=ax[]
    )
    jupyter.plot2d(
        integrator.
    )
    ax[0].set_title("")
    ax[1].set_title("")
    plt.tight_layout()
    plt.show()

    return integrator

In [ ]:
# set interface [widget, inline, nbagg]
%matplotlib widget

# define experimental parameters
E = 
c = 
h = 
wavelength = 
image_path = ""
calibrant_name = ""  # ["AgBh", "LaB6"] depends on calibrant used in experiment
detector_name = ""  # ["Pilatus1M", "PerkinElmer", "Eiger9M"] depends on detector used in experiment

# Set up the detector and calibrant
detector = pyFAI.detector_factory()
calibrant = CALIBRANT_FACTORY()
calibrant.wavelength = 

# Load the image data
img = 

# Initialize calibration & display interactive calibration widget
print(
    "",
    "Please follow the instructions to perform the calibration.",
    "1. Set the ring number (0-based value), below the plot",
    "2. Pick the ring by right-clicking with the mouse on the image.",
    "3. Restart at 1. for at least a second ring",
    "4. Click refine to launch the calibration.",
    sep="\n",
)
calibrated_image_object = Calibration(
    , , , 
)

# do not paste in further code beneath,
# because you first have to run the calibration process,
# before further calculate the results

In [ ]:
my_integrator = calibration_results()

---
### Exercise 2 - continuing

In **the second step**, we wish to load the diffraction data from a sample during a tensile test experiment. Start by loading both ``Mg10Gd_sample4_5_10minus5-00001.tif`` and ``Mg10Gd_sample4_5_10minus5-00060.tif`` and display at least one diffraction spectrum.

**Steps form above repeated:**

1. Display the detector image
2. Now perform the calibration as you did before.
3. Extract the calibrated geometry. Note that in this instance we will not perform any changes to the initial geometry, as no flight tube was used and we assume that a detector tilt is possible.
4. Create the azimuthal integrator as above.
5. Finish by displaying the integrated detector image. This is useful to check that you performed the calibration well. If you notice that the red lines are not coinciding with the major peaks you should go back to the calibration step.

---
#### Solution
_- solve the exercise beneath using markdown and/or code blocks -_

In [ ]:
sample1 = fabio.open("/Mg10Gd_sample4_5_10minus5-00001.tif").data
sample60 = fabio.open("/Mg10Gd_sample4_5_10minus5-00060.tif").data
jupyter.plot1d(my_integrator.integrate1d(, ))

You should be seeing three larger peaks at q-values between $20$ and $30\,\text{nm}^{-1}$. We now wish to fit the first of these three peaks and see how its position and width differ between the two images. Start by saving the azimuthal integrations in two arrays.

In [44]:
spectrum1 = my_integrator.integrate1d(, )
spectrum60 = 

To perform the fitting, it is useful to narrow down you data to that surrounding the peak (with no further peaks included). Therefore, generate arrays with contain only the peak we wish to fit and display both peaks. How do they differ visually?

In [ ]:
xdata1 = spectrum1[]
ydata1 = spectrum1[]

xdata1_peak = xdata1[() & ()]
ydata1_peak = 

xdata60 = 
ydata60 = 

xdata60_peak = 
ydata60_peak = 

fig, ax = plt.subplots(1, 2)
ax[0].plot(xdata1_peak, ydata1_peak)
ax[0].set_xlabel("Scattering vector q / nm$^{-1}$")
ax[0].set_title("Data 1")
ax[0].set_ylabel("Intensity")
ax[1].plot(xdata60_peak, ydata60_peak)
ax[1].set_xlabel("Scattering vector q / nm$^{-1}$")
ax[1].set_ylabel("Intensity")
ax[1].set_title("Data 60")
fig.tight_layout()
plt.show()

The peak for the `...s5-00060.tif` sample appears a little broader. In terms of the intensity, no differences are apparent.

In the **next step** define two functions: 
1. Define a **Gauss function** which will return the y-values of a Gaussian function given the x-values and the different parameters. Your Gauss function should take the form $$y=C+A\times e^{-\frac{(x-x_0)^2}{2\sigma^2}}$$
2. Define a **cost function** that computes the $L2$ **norm** of a given y-data and the Gaussian function from above with given x and parameter values

In [46]:
# define the gauss function
def gauss(x, params):
    y_g = 
    return y_g


# define a function that computes and returns the L2 loss
def cost_function(params, x, y):
    L2 = 
    return L2

We will **now fit two Gaussians** using the functions we have defined by using the **scipy function** [minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#minimize). Based on the visual appearance above, you should set initial values for the parameters of the Gaussian function. The output of the function contains the optimal parameter set. Using these, plot the two original peaks and the fitted Gaussians for a visual comparison. If your fit was successful, these should differ only slightly. 

In [ ]:
par0 = [1500, 10000, 22.5, 0.2]

output1 = minimize(, par0, args=(xdata1_peak, ydata1_peak))
fit_y1 = gauss(xdata1_peak, output1.x)

output60 = 
fit_y60 = 

fig, ax = plt.subplots(1, 2)
ax[0].plot(xdata1_peak, ydata1_peak, label="data")
ax[0].plot(xdata1_peak, fit_y1, linestyle="dashed", label="fit")
ax[0].set_xlabel("Scattering vector q / nm$^{-1}$")
ax[0].set_ylabel("Intensity")
ax[0].set_title("Data 1")
ax[0].legend()
ax[1].plot(xdata60_peak, ydata60_peak, label="data")
ax[1].plot(xdata60_peak, fit_y60, linestyle="dashed", label="fit")
ax[1].set_xlabel("Scattering vector q / nm$^{-1}$")
ax[1].set_ylabel("Intensity")
ax[1].set_title("Data 60")
ax[1].legend()
plt.show()

**Finally**, determine the **lattice spacing of the peaks** that you have fitted. Which **Mg-peak** are we assessing? How much do the lattice spacings and the FWHM (full width at half maximum) differ between the two measurements? How are you interpreting the results?

In [ ]:
d1_peak = 
print(
    "The peak position fitted for diffractogram 1 is", np.round(d1_peak, 4), "Angström."
)
fwhm1 = 
print("The full width at half maximum for diffractogram 1 is", np.round(fwhm1, 3))

d60_peak = 
print(
    "The peak position fitted for diffractogram 60 is",
    np.round(d60_peak, 4),
    "Angström.",
)
fwhm60 = 
print("The full width at half maximum for diffractogram 60 is", np.round(fwhm60, 3))

The fitted peak is the **Mg (100) peak**. The peak can be checked [here](http://rruff.geo.arizona.edu/AMS/amcsd.php). The peak position is differing only slightly, yet the FWHM (full width at half maximum) has changed significantly, which coincides with what we observed visually. The broadening of the peak is due to a strain that is introduced on the crystal lattice, as the experiment at hand is a tensile test. `...s5-00001.tif` is at **0 N** tensile load and `...s5-00060.tif` at **5.26 kN**.